SIMULATING MOLECULES

In [ ]:
#CREATE A RANDOM VECTOR

In [1]:
import numpy as np

np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

In [2]:
target_distr

array([0.51357006, 0.48642994])

In [ ]:
#We subsequently create a function that takes the parameters of our single U3 variational form as arguments and returns the corresponding quantum circuit:

In [5]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [ ]:
#Now we specify the objective function which takes as input a list of the variational form's parameters, and returns the cost associated with those parameters:

In [6]:
from qiskit import Aer, transpile, assemble

backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

In [ ]:
#Finally, we create an instance of the COBYLA optimizer, and run the algorithm. Note that the output varies from run to run. Moreover, while close, the obtained distribution might not be exactly the same as the target distribution, however, increasing the number of shots taken will increase the accuracy of the output.

In [8]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.517, 0.483]
Output Error (Manhattan Distance): 0.0006598812611608995
Parameters Found: [1.5332541  1.19679428 0.53597266]


In [ ]:
#Transpilation is the process of rewriting a given input circuit to match the topoplogy of a specific quantum device, and/or to optimize the circuit for execution on present day noisy quantum systems.

In [ ]:
#The circuits for RyRz corresponding to entanglement="full" and entanglement="linear" can be seen by executing the following code snippet:

In [12]:
from qiskit.circuit.library import EfficientSU2

entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = EfficientSU2(num_qubits=4, entanglement=entanglement)
    if entanglement == "linear":
        print("=============Linear Entanglement:=============")
    else:
        print("=============Full Entanglement:=============")
    # We initialize all parameters to 0 for this demonstration
    display(form.draw(fold=100))
    print()

=============Linear Entanglement:=============


┌──────────┐┌──────────┐     ┌──────────┐┌───────────┐                          ┌───────────┐»
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├───────────────────■──────┤ RY(θ[16]) ├»
     ├──────────┤├──────────┤┌─┴─┐└──────────┘└┬──────────┤┌───────────┐    ┌─┴─┐    └───────────┘»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├─────■───────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├────┤ X ├──────────■──────»
     ├──────────┤├──────────┤└───┘   ┌─┴─┐     └──────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    »
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├────────┤ X ├──────────■──────┤ RY(θ[10]) ├┤ RZ(θ[14]) ├────┤ X ├────»
     ├──────────┤├──────────┤        └───┘        ┌─┴─┐    ├───────────┤├───────────┤    └───┘    »
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├─────────────────────┤ X ├────┤ RY(θ[11]) ├┤ RZ(θ[15]) ├─────────────»
     └──────────┘└──────────┘                     └───┘    └───────────┘└───────────┘             »
«     ┌───────────┐                          ┌───────────┐┌───────────┐                          
«q_0: ┤ RZ(θ[20]) ├───────────────────■──────┤ RY(θ[24]) ├┤ RZ(θ[28]) ├──────────────────────────
«     ├───────────┤┌───────────┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐             
«q_1: ┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«     └───────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐
«q_2: ──────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├────┤ X ├──────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«         ┌─┴─┐    ├───────────┤├───────────┤    └───┘        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────────────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«         └───┘    └───────────┘└───────────┘                 └───┘    └───────────┘└───────────┘


=============Full Entanglement:=============


┌──────────┐┌──────────┐               ┌──────────┐┌───────────┐                         »
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■────■────■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├─────────────────────────»
     ├──────────┤├──────────┤┌─┴─┐  │    │  └──────────┘└───────────┘┌──────────┐┌───────────┐»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├──┼────┼───────■────────────■──────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├»
     ├──────────┤├──────────┤└───┘┌─┴─┐  │     ┌─┴─┐          │      └──────────┘├───────────┤»
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├─────┤ X ├──┼─────┤ X ├──────────┼───────────■──────┤ RY(θ[10]) ├»
     ├──────────┤├──────────┤     └───┘┌─┴─┐   └───┘        ┌─┴─┐       ┌─┴─┐    ├───────────┤»
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├──────────┤ X ├────────────────┤ X ├───────┤ X ├────┤ RY(θ[11]) ├»
     └──────────┘└──────────┘          └───┘                └───┘       └───┘    └───────────┘»
«                            ┌───────────┐┌───────────┐                                            »
«q_0: ──────■────────■────■──┤ RY(θ[16]) ├┤ RZ(θ[20]) ├────────────────────────────────■────────■──»
«         ┌─┴─┐      │    │  └───────────┘└───────────┘┌───────────┐┌───────────┐    ┌─┴─┐      │  »
«q_1: ────┤ X ├──────┼────┼────────■────────────■──────┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────┼──»
«     ┌───┴───┴───┐┌─┴─┐  │      ┌─┴─┐          │      └───────────┘├───────────┤┌───┴───┴───┐┌─┴─┐»
«q_2: ┤ RZ(θ[14]) ├┤ X ├──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├┤ X ├»
«     ├───────────┤└───┘┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤└───┘»
«q_3: ┤ RZ(θ[15]) ├─────┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────»
«     └───────────┘     └───┘                 └───┘        └───┘    └───────────┘└───────────┘     »
«          ┌───────────┐┌───────────┐                                       
«q_0: ──■──┤ RY(θ[24]) ├┤ RZ(θ[28]) ├───────────────────────────────────────
«       │  └───────────┘└───────────┘┌───────────┐┌───────────┐             
«q_1: ──┼────────■────────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«       │      ┌─┴─┐          │      └───────────┘├───────────┤┌───────────┐
«q_2: ──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«     ┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«     └───┘                 └───┘        └───┘    └───────────┘└───────────┘

VQE IMPLEMENTATION

In [4]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
#First, we define a function that takes an interatomic distance and returns the appropriate qubit operator, H, as well as some other information about the operator.

In [5]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift

In [6]:
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")

QiskitChemistryError: 'PySCF is not installed. See https://sunqm.github.io/pyscf/install.html'

GROUND STATE SOLVERS